In [49]:
import json
import pandas as pd
import sys
import os

from mapboxgl.viz import *
from mapboxgl.utils import *
import pysal.esda.mapclassify as mapclassify

## Set Mapbox token
Set a MAPBOX_ACCESS_TOKEN environment variable or copy/paste your token  
If you do not have a Mapbox access token, sign up for an account at https://www.mapbox.com/  
If you already have an account, you can grab your token at https://www.mapbox.com/account/  

In [50]:
# set Mapbox access token
token = os.getenv('MAPBOX_ACCESS_TOKEN')

## Get data
We're grabbing bike share station data from [Motivate](https://www.motivateco.com/), who runs the largest bike share programs in the US.  
The data below is specific to the [Divvy](https://www.divvybikes.com/) bike share program in Chicago, IL.  
You can access more bike share data from Motivate [here](https://www.motivateco.com/use-our-data/).

In [51]:
# load json data from the Divvy feed into a pandas dataframe
json_df = pd.read_json('https://feeds.divvybikes.com/stations/stations.json')
json_df.head(5)

,executionTime,stationBeanList
0,2018-02-25 07:30:37,"{u'status': u'IN_SERVICE', u'availableDocks': ..."
1,2018-02-25 07:30:37,"{u'status': u'IN_SERVICE', u'availableDocks': ..."
2,2018-02-25 07:30:37,"{u'status': u'IN_SERVICE', u'availableDocks': ..."
3,2018-02-25 07:30:37,"{u'status': u'IN_SERVICE', u'availableDocks': ..."
4,2018-02-25 07:30:37,"{u'status': u'IN_SERVICE', u'availableDocks': ..."


In [60]:
# parse dict in the stationBeanList column
df = json_df['stationBeanList'].apply(pd.Series)
df.head(5)

,altitude,availableBikes,availableDocks,city,id,is_renting,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,status,statusKey,statusValue,testStation,totalDocks
0,,9,18,Chicago,2,True,15541,2018-02-25 07:26:18,41.876393,,-87.620328,60605,Buckingham Fountain,,Buckingham Fountain,IN_SERVICE,1,In Service,False,27
1,,14,41,Chicago,3,True,15544,2018-02-25 07:28:59,41.867226,,-87.615355,,Shedd Aquarium,,Shedd Aquarium,IN_SERVICE,1,In Service,False,55
2,,1,21,Chicago,4,True,15545,2018-02-25 07:30:12,41.856268,,-87.613348,,Burnham Harbor,,Burnham Harbor,IN_SERVICE,1,In Service,False,23
3,,15,8,Chicago,5,True,13030,2018-02-25 07:29:20,41.874053,620 S. State St.,-87.627716,60605,State St & Harrison St,,State St & Harrison St,IN_SERVICE,1,In Service,False,23
4,,6,32,Chicago,6,True,15548,2018-02-25 07:27:07,41.885041,,-87.612794,60601,Dusable Harbor,,Dusable Harbor,IN_SERVICE,1,In Service,False,39


## Prepare data

In [54]:
# Clean up by dropping null rows
df.dropna(axis=1, how='all', inplace=True)
df.head(2)

,altitude,availableBikes,availableDocks,city,id,is_renting,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,status,statusKey,statusValue,testStation,totalDocks
0,,9,18,Chicago,2,True,15541,2018-02-25 07:26:18,41.876393,,-87.620328,60605,Buckingham Fountain,,Buckingham Fountain,IN_SERVICE,1,In Service,False,27
1,,14,41,Chicago,3,True,15544,2018-02-25 07:28:59,41.867226,,-87.615355,,Shedd Aquarium,,Shedd Aquarium,IN_SERVICE,1,In Service,False,55


## Visualize

In [61]:
# select properties
latitude_col = 'latitude'
longitude_col = 'longitude'
properties = ['stationName', 'availableBikes', 'availableDocks', 'totalDocks', 'statusValue']

In [62]:
# Create geojson data object
geo_json = df_to_geojson(df.fillna(''), 
                         lat=latitude_col, 
                         lon=longitude_col,
                         properties=properties,
                         precision=4
                        ) 

In [63]:
# Generate data breaks and color stops from colorBrewer
color_breaks = mapclassify.Natural_Breaks(df['availableBikes'], k=8, initial=0).bins
color_stops = create_color_stops(color_breaks, colors='Spectral')

# Generate radius breaks from data domain and circle-radius range
radius_breaks = mapclassify.Natural_Breaks(df["totalDocks"], k=8, initial=0).bins
radius_stops = create_radius_stops(radius_breaks, 1, 10)

# Create the viz
viz = GraduatedCircleViz(geo_json, 
                         access_token=token,
                         color_property = "availableBikes",
                         color_stops = color_stops,
                         radius_property = "totalDocks",
                         radius_stops = radius_stops,
                         center = (-87.6298, 41.8781),
                         zoom = 10,
                         below_layer = 'waterway-label')

viz.show()